# Histogram matching

This script takes in images and matches their pixel value histograms.

In [1]:
# some packages needed here
import cv2 as cv
import numpy as np
import glob
import os
from os import listdir
from os.path import isfile, join
from PIL import Image
from scipy import ndimage
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.exposure import match_histograms
import math

print("All libraries are loaded.")

All libraries are loaded.


## Input path

In [2]:
# give the input path for object images
parentPath = os.pardir
imagePath = glob.glob(parentPath + "/COCOimages/*.png")
imagePath.sort() # make sure the files are sorted

# count the number of images to process
nbrIm = len(imagePath)
print('There are', nbrIm, 'images to process.')

There are 80 images to process.


## Output path

In [3]:
# give the output path
outputPath = parentPath + r'/objects/'

# create the necessary output folder
if not os.path.exists(outputPath):
    os.makedirs(outputPath)

## Histogram matching

### Obtaining the average image histogram

First, read all images and extract the average pixel value histogram from them. Note the next cell could take a while to execute.

In [4]:
# calculating the average histogram of all objects

# creating an empty vector of all possible pixel values
pixelValues = np.zeros([256])

# adding the number of pixels contained in every object to the vector
for file in imagePath:

    im = cv.imread(file, -1)
    visible = im[:,:,3] == 255
    b, g, r, a = cv.split(im)
    im = b[visible] # only selecting the object part of the image

    for pixel in range(len(im)):
        pixel_value = int(im[pixel])
        pixelValues[pixel_value] += 1

# creating the empty array that will hold all the value points of the histogram
ref_array = np.empty(int(np.sum(pixelValues)))

# fill in the empty array
idx = 0

pixels = []
for x in range(256): pixels.append(x)

for i in pixels:
    ref_array[idx : idx + int(pixelValues[i])] = i
    idx += int(pixelValues[i])

# transform the 1d array obtained into a readable image
ref_array = np.reshape(ref_array, (1, len(ref_array))) # creating a 1D image from the vector
ref_im = ref_array.astype('uint8')

# get the average pixel colour
avg_colour = np.mean(ref_array)

### Equating image histograms

Next, read images, match their histogram to the average histogram, and write them. Note the next cell could take a while to execute.

In [6]:
for i in range(len(imagePath)):
    
    file = imagePath[i]
    name = os.path.splitext(os.path.basename(file))[0]
    
    im = cv.imread(file, -1)
    mask = ndimage.find_objects(im[:,:,3] > 0)[0] # find the bounding box containing the object
    im = im[mask] # only keep the object
    transparent = im[:,:,3] == 0

    # split the image to only equate the visible channels
    b, g, r, a = cv.split(im)
    b[transparent] = avg_colour
    
    b = match_histograms(b, ref_im)
    b = b.astype('uint8') # necessary for openCV
    
    # merge again to keep the transparency
    im = cv.merge([b, b, b, a])
    
    cv.imwrite(outputPath + name + '.png', im)
    
    print(i, 'images processed.')
    
print('It\'s all done, go get a coffee.')

0 images processed.
1 images processed.
2 images processed.
3 images processed.
4 images processed.
5 images processed.
6 images processed.
7 images processed.
8 images processed.
9 images processed.
10 images processed.
11 images processed.
12 images processed.
13 images processed.
14 images processed.
15 images processed.
16 images processed.
17 images processed.
18 images processed.
19 images processed.
20 images processed.
21 images processed.
22 images processed.
23 images processed.
24 images processed.
25 images processed.
26 images processed.
27 images processed.
28 images processed.
29 images processed.
30 images processed.
31 images processed.
32 images processed.
33 images processed.
34 images processed.
35 images processed.
36 images processed.
37 images processed.
38 images processed.
39 images processed.
40 images processed.
41 images processed.
42 images processed.
43 images processed.
44 images processed.
45 images processed.
46 images processed.
47 images processed.
48